In [1]:
from collections import namedtuple
from networkx import read_edgelist, set_node_attributes
from pandas import read_csv, Series
from numpy import array
from networkx import to_numpy_matrix, degree_centrality, betweenness_centrality, shortest_path_length
import mxnet.ndarray as nd
from sklearn.metrics import classification_report
from mxnet.gluon import HybridBlock

In [4]:
class SpectralRule(HybridBlock):
    def __init__(self, A, in_units, out_units, activation='relu', **kwargs):
        super().__init__(**kwargs)
        I = nd.eye(*A.shape)
        A_hat = A.copy() + I

        D = nd.sum(A_hat, axis=0)
        D_inv = D**-0.5
        D_inv = nd.diag(D_inv)

        A_hat = D_inv * A_hat * D_inv
        
        self.in_units, self.out_units = in_units, out_units
        
        with self.name_scope():
            self.A_hat = self.params.get_constant('A_hat', A_hat)
            self.W = self.params.get(
                'W', shape=(self.in_units, self.out_units)
            )
            if activation == 'identity':
                self.activation = lambda X: X
            else:
                self.activation = Activation(activation)

    def hybrid_forward(self, F, X, A_hat, W):
        aggregate = F.dot(A_hat, X)
        propagate = self.activation(
            F.dot(aggregate, W))
        return propagate

In [ ]:
hidden_layer_specs = [(4, 'tanh'), (2, 'tanh')] # Format: (units in layer, activation function)
in_units = in_units=X.shape[1]

features = HybridSequential()
with features.name_scope():
    for i, (layer_size, activation_func) in enumerate(hidden_layer_specs):
        layer = SpectralRule(
            A, in_units=in_units, out_units=layer_size, 
            activation=activation_func)
        features.add(layer)

        in_units = layer_size

In [2]:
import mxnet.ndarray as nd
from mxnet.gluon import HybridBlock, nn
from mxnet.gluon.nn import Activation
from mxnet.initializer import Uniform

In [3]:
def get_dict_values(data, keys):
    return [data[k] for k in keys]

In [62]:
class FeaturesTransform(HybridBlock):
    def __init__(self, entrylist, gene_to_index, entry_to_gene, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.layer_list = []
        with self.name_scope():
            for index, value in enumerate(entrylist):
                genelist = entry_to_gene[value].split(',')
                w = self.params.get(value, shape=(1, len(genelist)))
                self.layer_list.append((get_dict_values(gene_to_index, genelist),w))
                
            self.entry_b = self.params.get('entry_b', shape=(len(entrylist),1))
            
            if activation == 'identity':
                self.activation = lambda X: X
            else:
                self.activation = Activation(activation)
                
    def hybrid_forward(self, F, X, entry_b):
        # Change shape of b to comply with MXnet addition API
        contactlist = []
        for index, param in self.layer_list:
            contactlist.append(F.dot(param.data(), X[index,:]))
        y = nd.concat(*contactlist, dim=0) + entry_b
        return self.activation(y)

In [75]:
tmp = {'a': '1,2,3', 'b':'1,3,4', 'c':'2,1'}
gl = ['1', '2', '3', '4']
gl = dict(zip(gl, list(range(len(gl)))))
x = nd.array([[3,5,7,9],[3,5,7,9]]).reshape(4,2)
model = FeaturesTransform(tmp.keys(), gl, tmp, 'tanh')
model.initialize(Uniform(1))

In [76]:
res = model(x)

In [77]:
res


[[ 0.55201644  0.94704884]
 [-0.9999972  -1.        ]
 [-0.5160213   0.4451802 ]]
<NDArray 3x2 @cpu(0)>

In [27]:
res[0].concat(res[1], dim=0)

AttributeError: 'NDArray' object has no attribute 'concat'

In [41]:
get_dict_values(gl,('1','2'))

[0, 1]

In [21]:
model.layer_list

[Dense(None -> 1, linear), Dense(None -> 1, linear), Dense(None -> 1, linear)]